In [1]:
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-yankomagn')


Successfully saved authorization token.


In [2]:
# Create a map
Map = geemap.Map()

# Set map center
Map.setCenter(-69.91151172418182, 18.47803423428588, 12)
Map.add_basemap('SATELLITE')

# Load buildings and labels
buildings_uri_SD = '../data/0/overture/santodomingo_buildings.geojson'

In [3]:
# Add labels
labels = ee.FeatureCollection('projects/ee-yankomagn/assets/SD_labels')
labels = labels.map(lambda feature: feature.set('class', 1))
vislabelParams = {'color': 'brown'}
wards = labels.geometry().bounds()

# Other urban class for comparison
otherurban = ee.FeatureCollection('projects/ee-yankomagn/assets/SD_otherclass')
otherurban = otherurban.map(lambda feature: feature.set('class', 2))
visOtherUrbanParams = {'color': 'yellow'}

class_mask = ee.Image().paint(labels, 1).paint(otherurban, 2).unmask(0).rename('class')

# Load buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons')
buildings = buildings.filterBounds(wards)

Map.addLayer(buildings, {'color': '00FF00'}, 'Buildings')
Map.addLayer(wards, {}, 'Wards', False)
# labelmask_params = {'min': 0,'max': 1,'palette': ['blue', 'red']}
# Map.addLayer(class_mask, labelmask_params, 'Labels mask', False)

In [4]:
# Sentinel-2 multispectral imagery collection and processing
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
        .select(['B4', 'B3', 'B2', 'B8']) \
        .copyProperties(image, ["system:time_start"])

collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .median()

# Clip the image to the study area
image = collection.clip(wards)

# Add the Sentinel-2 layer to the map
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB', True)

# Calculate building density
empty = ee.Image().byte()
buildingsRec = buildings.map(lambda feature: feature.set({
    'area': ee.Number(1).divide(feature.geometry().area()).multiply(100),
    'const': 1
}))

buildingsRec_color = empty.paint(buildingsRec, 'area').unmask(0)

gaussian = ee.Kernel.gaussian(radius=60, units='meters', normalize=True, sigma=15)

density = buildingsRec_color.reduceNeighborhood(reducer=ee.Reducer.sum(), kernel=gaussian)
clippedDensity = density.clip(wards)
image = image.addBands(clippedDensity)

# Visualization parameters for density
palette = ['blue', 'cyan', 'green', 'yellow', 'red']
visParams = {
    'bands': ['constant_sum'],
    'palette': palette
}

Map.addLayer(image, visParams, 'Density', True)
Map.addLayer(labels, vislabelParams, 'Labels')

# Function to generate points within a feature collection
def generate_stratified_points(image, numPoints):
    stratified = image.addBands(ee.Image.pixelLonLat())
    points = stratified.stratifiedSample(
        numPoints=numPoints,
        classBand='class',
        region=wards,
        scale=30,
        geometries=True
    )
    return points

# Generate stratified sample points
sample_points = generate_stratified_points(class_mask, 2000)  # Adjust number as needed

# Split into training and validation
split = 0.7
training_points = sample_points.randomColumn().filter(ee.Filter.lt('random', split))
validation_points = sample_points.randomColumn().filter(ee.Filter.gte('random', split))

# Select the bands you want to use for classification
bands = ['B4', 'B3', 'B2', 'B8', 'constant_sum']

# Sample the image at the point locations
training = image.select(bands).addBands(class_mask).sampleRegions(
    collection=training_points,
    properties=['class'],
    scale=30
)

In [5]:
saved_classifier = ee.Classifier.load('projects/ee-yankomagn/assets/rf_classifier_trained')

In [8]:
# Train a Random Forest classifier
rf = ee.Classifier.smileRandomForest(100).train(
    features=training,
    classProperty='class',
    inputProperties=bands
)

# Apply the classifier to the image
classified = image.select(bands).classify(rf)

# Sample the classified image for validation
validation = classified.addBands(class_mask).sampleRegions(
    collection=validation_points,
    properties=['class'],
    scale=30
)


# Export the trained classifier as an asset
# export_path = 'projects/ee-yankomagn/assets/rf_classifier_trained' # 'users/your_username/rf_classifier'

# export_task = ee.batch.Export.table.toAsset(
#     collection=ee.FeatureCollection([ee.Feature(None, rf.explain())]),
#     description='Export RF Classifier',
#     assetId=export_path
# )

# # Start the export task
# export_task.start()

In [9]:
# Compute accuracy
testAccuracy = validation.errorMatrix('class', 'classification')

# Use evaluate() instead of getInfo()
accuracy_result = testAccuracy.accuracy().evaluate()
print('Overall accuracy: ', accuracy_result['accuracy'])

AttributeError: 'Number' object has no attribute 'evaluate'

In [10]:
# Compute class-specific accuracy
class_accuracy = testAccuracy.producersAccuracy().evaluate()
print('Class-specific accuracy: ', class_accuracy['accuracy'])

AttributeError: 'Array' object has no attribute 'evaluate'

In [11]:
# If you want to focus on the accuracy of your main class of interest (class 1)
main_class_accuracy = testAccuracy.producersAccuracy().get([1]).evaluate()
print('Accuracy for main class of interest: ', main_class_accuracy['accuracy'])

AttributeError: 'Number' object has no attribute 'evaluate'

In [12]:
# Visualize the classification result
class_palette = ['black', 'red', 'blue']  # black for background, red for labels, blue for other urban
class_vis = {
    'min': 0,
    'max': 2,
    'palette': class_palette
}
Map.addLayer(classified, class_vis, 'Classification')

# Display the map
Map

EEException: Computation timed out.